In [20]:
import os
import spacy
import cluegen
import numpy as np

from nltk.corpus import wordnet
#import nltk
#nltk.download('wordnet')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [251]:
# with open(os.path.join(path, 'google-10000-english-no-swears.txt'), 'r') as f:
#     for line in f:
#         token = line.split('\t')[0]
#         lemma = nlp(token)
#         # filter out stop words and only keep nouns, verbs and adjectives
#         if lemma.has_vector and not lemma[0].is_stop and lemma[0].pos_ in ['NOUN', 'VERB', 'ADJ']:
#             all_clues.append(lemma)
#             if lemma[0].pos_ == 'NOUN':
#                 num_nouns += 1
#             elif lemma[0].pos_ == 'VERB':
#                 num_verbs += 1
#             elif lemma[0].pos_ == 'ADJ':
#                 num_adjs += 1

# print(len(all_clues))
# print(num_nouns, 'nouns')
# print(num_verbs, 'verbs')
# print(num_adjs, 'adjectives')


# open filtered clues
all_clues = []
lemmas = set()
with open('filtered_clues.txt', 'r') as f:
    for line in f:
        token = line.strip()
        lemma = nlp(token)
        if lemma.text not in lemmas:
            lemmas.add(lemma.text)
            all_clues.append(lemma)

In [331]:
# # remove any words that are not nouns, verbs, or adjectives
# all_clues = [w for w in all_clues if w[0].pos_ in ['NOUN', 'VERB', 'ADJ']]

# # save filtered clues to a new text file
# with open('filtered_clues.txt', 'w') as f:
#     for clue in all_clues:
#         f.write(clue.text + '\n')

# # words from https://github.com/first20hours/google-10000-english
# import pickle
# with open('filtered_clues.pkl', 'wb') as f:
#     pickle.dump(all_clues, f)
# len(all_clues)

63202

In [95]:
path = './'

nlp = spacy.load('en_core_web_lg')

# all_clues = []
# num_nouns = 0
# num_verbs = 0
# num_adjs = 0
# with open(os.path.join(path, 'top_words.txt'), 'r') as f:
#     for line in f:
#         token = line.strip()
#         lemma = nlp(token)
#         # filter out stop words and only keep nouns, verbs and adjectives
#         if lemma.has_vector and not lemma[0].is_stop and lemma[0].pos_ in ['NOUN', 'VERB', 'ADJ']:
#             all_clues.append(lemma)
#             if lemma[0].pos_ == 'NOUN':
#                 num_nouns += 1
#             elif lemma[0].pos_ == 'VERB':
#                 num_verbs += 1
#             elif lemma[0].pos_ == 'ADJ':
#                 num_adjs += 1
# print(len(all_clues))
# print(num_nouns, 'nouns')
# print(num_verbs, 'verbs')
# print(num_adjs, 'adjectives')

2477
1293 nouns
796 verbs
388 adjectives


In [349]:
clue_generator = cluegen.ClueGenerator(path=path)

In [366]:
# make a list of 25 words
my_words = ['Africa', 'ring', 'dog', 'Earth', 'piano', 
         'apple', 'chair', 'table', 'book', 'pen', 
         'computer', 'phone', 'guitar', 'car', 'house', 
         'tree', 'flower', 'water', 'air', 'fire', 
         'sun', 'moon', 'star', 'sky', 'cloud']
your_words = ['ocean', 'mountain', 'river', 'lake', 'road',
               'bridge', 'building', 'city', 'country', 'continent',
               'planet', 'universe', 'galaxy', 'solar', 'fish']

clusters, clues = clue_generator.generate(my_words)
for cluster, clue in zip(clusters, clues):
    print(cluster, '-->', clue)

['piano', 'guitar'] --> ['song' 'band' 'bass']
['moon', 'sky'] --> ['sun' 'sea' 'air']
['sun', 'moon', 'sky'] --> ['sea' 'air' 'eye']
['tree', 'flower'] --> ['sun' 'oil' 'wood']
['water', 'air'] --> ['gas' 'oil' 'sea']
['ring', 'table'] --> ['ions' 'low' 'use']
['computer', 'phone'] --> ['app' 'web' 'user']
['car', 'house'] --> ['bus' 'room' 'homes']
['apple', 'tree', 'flower'] --> ['oil' 'tea' 'sun']
['fire', 'water', 'air'] --> ['gas' 'oil' 'sea']
['dog', 'car', 'house'] --> ['pet' 'room' 'bus']
['book', 'pen'] --> ['ink' 'art' 'web']
['Earth', 'cloud'] --> ['sky' 'sun' 'air']


In [119]:
clusters = clue_generator.cluster_words(words)
for cluster in clusters:
    print(np.round(cluster.distance, 2), cluster.get_lemmas())

0.4 [piano, guitar]
0.53 [moon, sky]
0.6 [sun, moon, sky]
0.69 [tree, flower]
0.75 [water, air]
0.88 [ring, table]
0.9 [computer, phone]
0.94 [car, house]
0.99 [apple, tree, flower]
1.03 [fire, water, air]
1.17 [dog, car, house]
1.19 [book, pen]
1.2 [Earth, cloud]
1.3 [chair, dog, car, house]
1.33 [ring, table, apple, tree, flower]
1.41 [sun, moon, sky, Earth, cloud]
1.46 [Africa, star]
1.49 [computer, phone, book, pen]


In [58]:
def cosine_dist(v1, v2):
    return 1 - (np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))

In [309]:
# TODO: improve the clue generation process

# 1) make a better list of possible clue words
#    (e.g. only nouns and some verbs and adjectives)
# 2) get the top few clues to a word
# 3) then, check those words hypernyms for similarity as well
#    return the most similar of the possibilities

cluster = clusters[0]
cluster_words = [c.text.lower() for c in cluster.get_lemmas()]

print('Cluster')
print(cluster.get_lemmas())

def no_word_overlap(word, words):
    # make sure the word does not equal,
    # or is not a substring of any of the words, and vice versa
    return (word not in words) and (not any([word in w for w in words]) and (not any([w in word for w in words])))

all_clues_less_cluster = []
for clue_lemma in all_clues:
    clue_word = clue_lemma.text.lower()
    if no_word_overlap(clue_word, cluster_words):
        all_clues_less_cluster.append(clue_lemma)

# get the average vector for the cluster
cluster_avg = np.mean([w.vector for w in cluster.get_lemmas()], axis=0)
# get the distance from each word to the cluster average using cosine distance
#distances = [np.linalg.norm(w.vector - cluster_avg) for w in all_clues_less_cluster]
distances = [cosine_dist(w.vector, cluster_avg) for w in all_clues_less_cluster]
# get the index of the word with the smallest distance
clue_index = np.argmin(distances)
# return the top 10 words with the smallest distance
clues = [(all_clues_less_cluster[i], distances[i]) for i in np.argsort(distances)[:10]]
for clue, dist, in clues:
    print(np.round(dist, 2), clue, end=' ')
    clue_wn = wordnet.synsets(clue.text)
    if len(clue_wn) > 0:
        clue_wn = clue_wn[0] # get the most common synset
    else:
        continue
    
    for hyper in clue_wn.hypernyms():
        hyper_word = hyper.name().split('.')[0]
        hyper_lemma = nlp(hyper_word)
        if hyper_lemma.has_vector:
            hyper_dist = cosine_dist(nlp(hyper_word).vector, cluster_avg)
            print('['+str(np.round(hyper_dist, 2)), hyper_word, end='] ')
    for hypo in clue_wn.hyponyms():
        hypo_word = hypo.name().split('.')[0]
        hypo_lemma = nlp(hypo_word)
        if hypo_lemma.has_vector:
            hypo_dist = cosine_dist(nlp(hypo_word).vector, cluster_avg)
            print('['+str(np.round(hypo_dist, 2)), hypo_word, end='] ')
    print()

Cluster
[piano, guitar]
0.17 violin 
0.2 saxophone [0.86 single-reed_instrument] 
0.22 harpsichord [0.41 clavier] [0.66 spinet] [0.92 virginal] 
0.23 banjo 
0.24 trombone [0.57 brass] [0.91 sackbut] 
0.24 mandolin [0.55 mandola] 
0.24 bass [0.73 pitch] 
0.26 drumset 0.26 solos [1.03 activity] 
0.26 banjos 


In [265]:
clusters = clue_generator.cluster_words(my_words)
for cluster in clusters:
    print(np.round(cluster.distance, 2), cluster.get_lemmas())

0.4 [piano, guitar]
0.53 [moon, sky]
0.6 [sun, moon, sky]
0.69 [tree, flower]
0.75 [water, air]
0.88 [ring, table]
0.9 [computer, phone]
0.94 [car, house]
0.99 [apple, tree, flower]
1.03 [fire, water, air]
1.17 [dog, car, house]
1.19 [book, pen]
1.2 [Earth, cloud]
1.3 [chair, dog, car, house]
1.33 [ring, table, apple, tree, flower]
1.41 [sun, moon, sky, Earth, cloud]
1.46 [Africa, star]
1.49 [computer, phone, book, pen]


In [267]:
cluster_num = 0

In [340]:
all_clues = []
with open('filtered_clues.txt', 'r') as f:
    for line in f:
        token = line.strip()
        lemma = nlp(token)
        if lemma.has_vector:
            all_clues.append(lemma)

In [348]:
import numpy as np
import pandas as pd

# 1) create an array for the word vectors and an array for the words
#    to speed up calculations
all_clue_words = np.array([w.text for w in all_clues])
all_clue_vectors = np.array([w.vector for w in all_clues])

# 2) save the all_clue_vectors array to a numpy object
np.save('all_clue_words.npy', all_clue_words)
#np.save('all_clue_vectors.npy', all_clue_vectors)


# 3) save the all_clue_words and all_clue_vectors arrays to a CSV file
#df = pd.DataFrame({'words': all_clue_words, 'vectors': all_clue_vectors.tolist()})
#df.to_csv('all_clue_words_vectors.csv', index=False)

In [346]:
#cluster_words = ['space', 'star', 'moon']
#cluster_words = [str(w) for w in np.random.choice(all_clue_words, 3, replace=False)]
cluster_words = ['cloud', 'sky', 'fly', 'fish']
my_words = cluster_words
your_words = ['cloud', 'sky', 'fly', 'fish']
cluster_vectors = np.array([nlp(w).vector for w in cluster_words])
your_vectors = np.array([nlp(w).vector for w in your_words])


# 2) find a word that is closest to the average vector of the cluster
#    and farthest from the other words outside the cluster
def no_word_overlap(word, words):
    # make sure the word does not equal,
    # or is not a substring of any of the words, and vice versa
    return (word not in words) and (not any([word in w for w in words]) and (not any([w in word for w in words])))

# filter out clues that overlap with the words
possible_clues = []
possible_vectors = []
for clue in all_clues:
    if no_word_overlap(clue.text, my_words) and no_word_overlap(clue.text, your_words):
        possible_clues.append(clue)
        possible_vectors.append(clue.vector)
possible_clues = np.array(possible_clues)
possible_vectors = np.array(possible_vectors)

print('possible clues:', len(possible_clues))


import matplotlib.pyplot as plt
# calculate the cosine distance between each clue and the average vector
cluster_dists = possible_vectors @ cluster_vectors.T
#cluster_dists = possible_vectors @ np.expand_dims(np.mean(cluster_vectors, axis=0), 0).T
your_dists = possible_vectors @ your_vectors.T
# normalize the dists
cluster_dists = 1 - cluster_dists / np.expand_dims(np.linalg.norm(cluster_vectors, axis=1),0) * np.expand_dims(np.linalg.norm(possible_vectors, axis=1), 1)
your_dists = 1 - your_dists / np.expand_dims(np.linalg.norm(your_vectors, axis=1),0) * np.expand_dims(np.linalg.norm(possible_vectors, axis=1), 1)

best_dists = np.sum(cluster_dists, axis=1) # - np.sum(your_dists, axis=1)
print(best_dists.shape)

clue_idxs = np.argsort(best_dists)[:100]
#print(clue_idxs[:100])
cluster_words, possible_clues[clue_idxs] #, best_dists[clue_idxs]

possible clues: 29943
(29943,)


/var/folders/rc/lxb7w_kn1gd1gppq2y1hq2tm0000gn/T/ipykernel_49494/1887027835.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  possible_clues = np.array(possible_clues)


(['cloud', 'sky', 'fly', 'fish'],
 array([sea, sun, air, eye, ice, ash, red, wind, jet, rain, oil, dry, snow,
        light, water, low, gas, land, winds, waters, tree, sand, seas,
        lake, mid, day, use, heat, fire, trees, moon, eat, birds, bird,
        ray, prey, way, skin, eyes, egg, dust, ocean, grow, flow, end,
        soil, tail, hot, map, web, ball, feet, rod, big, wood, edge, lot,
        ship, boat, blue, nest, food, flight, storm, dark, river, hole,
        thin, lakes, ink, skies, bat, plane, mud, bed, fat, sail, thing,
        wet, wings, rains, bit, sands, cut, rock, area, space, rays, car,
        waves, ions, reef, roof, body, rocks, eggs, dog, wing, night,
        areas], dtype=object))